# 1. MODULO

In [ ]:
# Importa el módulo 'os' para interactuar con el sistema de archivos y rutas del sistema operativo
import os

# Importa 'pandas' como 'pd', una biblioteca potente para manipulación y análisis de datos mediante DataFrames
import pandas as pd

# Importa el módulo 'datetime' para trabajar con fechas y horas de manera eficiente
import datetime

# 2. Unificación anual maximo y minimos maestros

In [ ]:
import os
import pandas as pd
from datetime import datetime

# Definir las rutas de entrada y salida
R_Maestros = r"C:\Users\osmarrincon\OneDrive - uniminuto.edu\Capresoca\AlmostClear\Procesos BDUA\Subsidiados\Maestro\MS"
R_SALIDA_ANUAL = os.path.join(R_Maestros, "All")

# Asegurar que las carpetas de salida existen
os.makedirs(os.path.join(R_SALIDA_ANUAL, "Max"), exist_ok=True)
os.makedirs(os.path.join(R_SALIDA_ANUAL, "Min"), exist_ok=True)

# Lista de encabezados proporcionados (43 columnas)
encabezados = [
    "AFL_ID", "ENT_ID", "TPS_IDN_ID_CF", "HST_IDN_NUMERO_IDENTIFICACION_CF", "TPS_IDN_ID",
    "HST_IDN_NUMERO_IDENTIFICACION", "AFL_PRIMER_APELLIDO", "AFL_SEGUNDO_APELLIDO",
    "AFL_PRIMER_NOMBRE", "AFL_SEGUNDO_NOMBRE", "AFL_FECHA_NACIMIENTO", "TPS_GNR_ID",
    "AFL_PAIS_NACIMIENTO", "AFL_MUNICIPIO_NACIMIENTO", "AFL_NACIONALIDAD",
    "AFL_SEXO_IDENTIFICACION", "AFL_DISCAPACIDAD", "TPS_AFL_ID", "TPS_PRN_ID",
    "TPS_GRP_PBL_ID", "TPS_NVL_SSB_ID", "NUMEROFICHASISBEN", "TPS_CND_BNF_ID", "DPR_ID",
    "MNC_ID", "ZNS_ID", "AFL_FECHA_AFILIACION_SGSSS", "AFC_FECHA_INICIO", "NUMERO CONTRATO",
    "FECHADE INICIO DEL CONTRATO", "CNT_AFL_TPS_GRP_PBL_ID", "CNT_AFL_TPS_PRT_ETN_ID",
    "TPS_MDL_SBS_ID", "TPS_EST_AFL_ID", "CND_AFL_FECHA_INICIO", "CND_AFL_FECHA_INICIO_2",
    "GRP_FML_COTIZANTE_ID", "PORTABILIDAD", "COD_IPS_P", "MTDLG_G_P", "SUB_SISBEN_IV",
    "MARCASISBENIV+MARCASISBENIII", "CRUCE_BDEX_RNEC",
]

def get_maestro_date(file_name):
    """Extrae la fecha del nombre del archivo y la devuelve como objeto datetime."""
    try:
        idx = file_name.upper().find('EPS025MS00')
        if idx != -1:
            fecha_str = file_name[idx + 10:idx + 18]
            return datetime.strptime(fecha_str, '%d%m%Y')
    except (ValueError, IndexError):
        pass
    return None

def pad_dataframe(df, target_cols):
    """Ajusta las columnas del DataFrame a la estructura de 43 columnas."""
    num_cols = df.shape[1]
    if num_cols == 31:
        for i in range(5): df.insert(12, f'col_vacia_{12+i}', '')
        for i in range(7): df[f'col_vacia_end_{i}'] = ''
    elif num_cols == 37:
        for i in range(5): df.insert(12, f'col_vacia_{12+i}', '')
        df['col_vacia_end_0'] = ''
    elif num_cols == 38:
        for i in range(5): df.insert(12, f'col_vacia_{12+i}', '')
    
    if df.shape[1] == len(target_cols):
        df.columns = target_cols
    else:
        print(f"Error: El DataFrame resultante no tiene 43 columnas. Tiene: {df.shape[1]}")
    return df

# Recorrer cada subdirectorio de año
for dir_name in os.listdir(R_Maestros):
    year_path = os.path.join(R_Maestros, dir_name)
    if os.path.isdir(year_path) and dir_name != "All":
        print(f"--- Procesando carpeta: {dir_name} ---")
        
        monthly_files = {}
        for root, _, files in os.walk(year_path):
            for file in files:
                if file.lower().endswith('.txt'):
                    file_path = os.path.join(root, file)
                    file_date = get_maestro_date(file)
                    
                    if file_date:
                        month_year = file_date.strftime('%Y-%m')
                        if month_year not in monthly_files:
                            monthly_files[month_year] = []
                        monthly_files[month_year].append({'path': file_path, 'date': file_date, 'name': file})

        if not monthly_files:
            print("No se encontraron archivos válidos. Saltando.")
            continue

        output_min_path = os.path.join(R_SALIDA_ANUAL, "Min", f"{dir_name}_Min.txt")
        output_max_path = os.path.join(R_SALIDA_ANUAL, "Max", f"{dir_name}_Max.txt")

        # Proceso optimizado: escribir directamente a los archivos de salida
        first_min_file = True
        first_max_file = True

        for month_year in sorted(monthly_files.keys()):
            files_list = sorted(monthly_files[month_year], key=lambda x: x['date'])
            
            # Procesar archivo mínimo
            min_file_info = files_list[0]
            df_min = pd.read_csv(min_file_info['path'], sep=",", header=None, encoding='latin1', dtype=str)
            df_min = pad_dataframe(df_min, encabezados)
            df_min['nombre_archivo'] = min_file_info['name']
            df_min['fecha_maestro'] = min_file_info['date'].strftime('%d/%m/%Y')
            
            # Escribir al archivo de salida
            df_min.to_csv(output_min_path, sep=",", index=False, encoding='latin1', header=first_min_file, mode='a')
            if first_min_file: first_min_file = False

            # Procesar archivo máximo
            max_file_info = files_list[-1]
            df_max = pd.read_csv(max_file_info['path'], sep=",", header=None, encoding='latin1', dtype=str)
            df_max = pad_dataframe(df_max, encabezados)
            df_max['nombre_archivo'] = max_file_info['name']
            df_max['fecha_maestro'] = max_file_info['date'].strftime('%d/%m/%Y')
            
            # Escribir al archivo de salida
            df_max.to_csv(output_max_path, sep=",", index=False, encoding='latin1', header=first_max_file, mode='a')
            if first_max_file: first_max_file = False
            
        print(f"Archivos consolidados para {dir_name} guardados en 'All/Min' y 'All/Max'.")

print("\nProceso de consolidación anual finalizado.")

# 3. validar maximos y minimos

In [ ]:
import os
import pandas as pd
from datetime import datetime

# Definir las rutas de entrada y salida
R_Maestros = r"C:\Users\osmarrincon\OneDrive - uniminuto.edu\Capresoca\AlmostClear\Procesos BDUA\Subsidiados\Maestro\MS"
ruta_maximos_anual = os.path.join(R_Maestros, "All", "Max")
ruta_minimos_anual = os.path.join(R_Maestros, "All", "Min")
ruta_salida = os.path.join(R_Maestros, "All")

def validar_y_corregir_optimo(ruta_directorio, es_maximo):
    """
    Valida y corrige los archivos anuales de manera eficiente en dos pases.
    """
    grupo_nombre = "Máximos" if es_maximo else "Mínimos"
    print(f"--- Iniciando validación y corrección optimizada de {grupo_nombre} ---")

    # --- Primer Pase: Consolidar solo metadatos ---
    print("Paso 1: Consolidando metadatos (fechas y nombres de archivo)...")
    dfs_metadata = []
    if not os.path.exists(ruta_directorio):
        print(f"Advertencia: La ruta no existe: {ruta_directorio}")
        return
        
    archivos = [f for f in os.listdir(ruta_directorio) if f.endswith('.txt')]
    for file_name in archivos:
        file_path = os.path.join(ruta_directorio, file_name)
        try:
            # Leer solo las columnas de interés
            df_temp = pd.read_csv(file_path, sep=",", encoding='latin1', usecols=['fecha_maestro', 'nombre_archivo'], dtype=str)
            df_temp['año_consolidado'] = file_name.split('_')[0]
            dfs_metadata.append(df_temp)
        except Exception as e:
            print(f"Advertencia: Error al leer metadatos de {file_name}: {e}")
    
    if not dfs_metadata:
        print(f"No se encontraron metadatos para el grupo {grupo_nombre}.")
        return
    
    metadata_df = pd.concat(dfs_metadata, ignore_index=True)
    
    # --- CORRECCIÓN DEL ERROR ---
    # Eliminar las filas donde el valor de la fecha es el encabezado literal
    metadata_df = metadata_df[metadata_df['fecha_maestro'] != 'fecha_maestro']
    
    # Ahora sí, convertir a formato de fecha
    metadata_df['fecha_maestro'] = pd.to_datetime(metadata_df['fecha_maestro'], format='%d/%m/%Y')
    metadata_df['mes_año'] = metadata_df['fecha_maestro'].dt.to_period('M')

    # --- Validación: Identificar los maestros correctos ---
    print("Paso 2: Identificando el maestro correcto para cada mes...")
    if es_maximo:
        idx_to_keep = metadata_df.loc[metadata_df.groupby('mes_año')['fecha_maestro'].idxmax()].index
    else:
        idx_to_keep = metadata_df.loc[metadata_df.groupby('mes_año')['fecha_maestro'].idxmin()].index

    maestros_correctos = set(metadata_df.loc[idx_to_keep, 'nombre_archivo'])

    # --- Segundo Pase: Corregir y sobrescribir archivos uno por uno ---
    print("Paso 3: Sobrescribiendo archivos con los registros correctos...")
    
    df_validado = metadata_df.loc[idx_to_keep].copy()
    
    for file_name in archivos:
        file_path = os.path.join(ruta_directorio, file_name)
        try:
            df_completo = pd.read_csv(file_path, sep=",", encoding='latin1', dtype=str)
            df_filtrado = df_completo[df_completo['nombre_archivo'].isin(maestros_correctos)].copy()
            df_filtrado.to_csv(file_path, sep=",", index=False, encoding='latin1')
            print(f"  - Archivo corregido: {file_name}")
            
        except Exception as e:
            print(f"Advertencia: Error al corregir {file_name}: {e}")

    # --- Verificación de la completitud de los meses ---
    print("\n--- Verificando la completitud de los meses ---")
    años_consolidados = sorted(df_validado['año_consolidado'].unique())
    año_actual = datetime.now().year
    mes_actual = datetime.now().month

    for año_consolidado in años_consolidados:
        try:
            año_val = int(año_consolidado)
        except ValueError:
            año_val = int(año_consolidado.split('-')[0])
        
        meses_disponibles = sorted(df_validado[df_validado['año_consolidado'] == año_consolidado]['mes_año'].dt.month.unique())
        
        if año_val == año_actual:
            meses_esperados = list(range(1, mes_actual + 1))
        else:
            meses_esperados = list(range(1, 13))

        meses_faltantes = [m for m in meses_esperados if m not in meses_disponibles]
        
        if meses_faltantes:
            print(f"Advertencia para {año_consolidado}: Faltan maestros para los meses: {meses_faltantes}")
        else:
            print(f"Validación exitosa para {año_consolidado}: Se encontraron maestros para todos los meses esperados.")

    print(f"\nProceso de validación y corrección de {grupo_nombre} finalizado.")

# Ejecutar el proceso para máximos y mínimos
validar_y_corregir_optimo(ruta_maximos_anual, es_maximo=True)
validar_y_corregir_optimo(ruta_minimos_anual, es_maximo=False)

print("\nTodos los maestros anuales han sido validados, corregidos y están listos para la unificación total.")

# 4. Unicos Por año

In [4]:
import os
import pandas as pd

# Definir la ruta de la carpeta All, que contiene los subdirectorios Max y Min
ruta_base_all = r"C:\Users\osmarrincon\OneDrive - uniminuto.edu\Capresoca\AlmostClear\Procesos BDUA\Subsidiados\Maestro\MS\All"

def consolidar_por_año(ruta_directorio):
    """
    Agrupa los archivos por su año base y los consolida en un único archivo por año.
    """
    grupo_nombre = "Máximos" if "Max" in ruta_directorio else "Mínimos"
    print(f"--- Iniciando unificación anual para el grupo de {grupo_nombre} ---")

    if not os.path.exists(ruta_directorio):
        print(f"Advertencia: La ruta no existe: {ruta_directorio}")
        return

    # Diccionario para agrupar archivos por su año base
    archivos_por_año = {}
    archivos_a_eliminar = []

    for file_name in os.listdir(ruta_directorio):
        if file_name.endswith('.txt'):
            year_prefix = file_name.split('_')[0]
            # Extraer el año base (ej. '2022' de '2022-01')
            base_year = year_prefix.split('-')[0]
            
            if base_year not in archivos_por_año:
                archivos_por_año[base_year] = []
            
            file_path = os.path.join(ruta_directorio, file_name)
            archivos_por_año[base_year].append(file_path)
            archivos_a_eliminar.append(file_path)

    for base_year, file_paths in archivos_por_año.items():
        print(f"  - Consolidando archivos para el año {base_year}...")
        dfs = []
        for file_path in file_paths:
            try:
                df = pd.read_csv(file_path, sep=",", encoding='latin1', dtype=str)
                dfs.append(df)
            except Exception as e:
                print(f"    - Advertencia: No se pudo leer {os.path.basename(file_path)}: {e}")
        
        if dfs:
            df_unificado = pd.concat(dfs, ignore_index=True)
            output_file_name = f"{base_year}_{grupo_nombre.replace('os','')}.txt"
            output_path = os.path.join(ruta_directorio, output_file_name)
            
            df_unificado.to_csv(output_path, sep=",", index=False, encoding='latin1')
            print(f"  - Archivo unificado guardado: {output_file_name}")

    # Eliminar los archivos originales después de la unificación
    print("\n  - Eliminando archivos originales...")
    for file_path in archivos_a_eliminar:
        try:
            os.remove(file_path)
        except Exception as e:
            print(f"    - Advertencia: No se pudo eliminar {os.path.basename(file_path)}: {e}")
    
    print(f"--- Proceso de unificación anual de {grupo_nombre} finalizado. ---\n")

# Ejecutar el proceso para ambos grupos
consolidar_por_año(os.path.join(ruta_base_all, "Max"))
consolidar_por_año(os.path.join(ruta_base_all, "Min"))

  - Archivo unificado guardado: 2019_Mínim.txt
  - Consolidando archivos para el año 2020...
  - Archivo unificado guardado: 2020_Mínim.txt
  - Consolidando archivos para el año 2021...
  - Archivo unificado guardado: 2021_Mínim.txt
  - Consolidando archivos para el año 2022...
  - Archivo unificado guardado: 2022_Mínim.txt
  - Consolidando archivos para el año 2023...
  - Archivo unificado guardado: 2023_Mínim.txt
  - Consolidando archivos para el año 2024...
  - Archivo unificado guardado: 2024_Mínim.txt
  - Consolidando archivos para el año 2025...
  - Archivo unificado guardado: 2025_Mínim.txt

  - Eliminando archivos originales...
--- Proceso de unificación anual de Mínimos finalizado. ---



# 5. Consolidado Maximo y minimo

In [5]:
import os

# Definir las rutas de entrada y salida
R_Maestros = r"C:\Users\osmarrincon\OneDrive - uniminuto.edu\Capresoca\AlmostClear\Procesos BDUA\Subsidiados\Maestro\MS"
ruta_maximos_anual = os.path.join(R_Maestros, "All", "Max")
ruta_minimos_anual = os.path.join(R_Maestros, "All", "Min")
ruta_salida = R_Maestros

# --- Unificación de archivos máximos ---
print("---")
print("Iniciando la unificación de maestros máximos de todos los años...")

archivos_maximos = [f for f in sorted(os.listdir(ruta_maximos_anual)) if f.endswith('.txt')]

if archivos_maximos:
    output_path_max = os.path.join(ruta_salida, "MS_Unif_Maximo.txt")
    with open(output_path_max, 'w', encoding='latin1') as outfile:
        
        # Leer el primer archivo para escribir los encabezados una sola vez
        with open(os.path.join(ruta_maximos_anual, archivos_maximos[0]), 'r', encoding='latin1') as infile:
            header = infile.readline()
            outfile.write(header)

        # Recorrer todos los archivos para escribir los datos sin encabezados
        for file_name in archivos_maximos:
            file_path = os.path.join(ruta_maximos_anual, file_name)
            with open(file_path, 'r', encoding='latin1') as infile:
                next(infile) # Omitir el encabezado de este archivo
                for line in infile:
                    outfile.write(line)

    print("\nProceso de máximos completado.")
    print(f"Archivo unificado guardado en: {output_path_max}")
else:
    print("\nNo se encontraron archivos de maestros máximos para unificar.")

# --- Unificación de archivos mínimos ---
print("---")
print("Iniciando la unificación de maestros mínimos de todos los años...")

archivos_minimos = [f for f in sorted(os.listdir(ruta_minimos_anual)) if f.endswith('.txt')]

if archivos_minimos:
    output_path_min = os.path.join(ruta_salida, "MS_Unif_Minimo.txt")
    with open(output_path_min, 'w', encoding='latin1') as outfile:
        
        # Leer el primer archivo para escribir los encabezados una sola vez
        with open(os.path.join(ruta_minimos_anual, archivos_minimos[0]), 'r', encoding='latin1') as infile:
            header = infile.readline()
            outfile.write(header)

        # Recorrer todos los archivos para escribir los datos sin encabezados
        for file_name in archivos_minimos:
            file_path = os.path.join(ruta_minimos_anual, file_name)
            with open(file_path, 'r', encoding='latin1') as infile:
                next(infile) # Omitir el encabezado de este archivo
                for line in infile:
                    outfile.write(line)

    print("\nProceso de mínimos completado.")
    print(f"Archivo unificado guardado en: {output_path_min}")
else:
    print("\nNo se encontraron archivos de maestros mínimos para unificar.")

print("---")
print("Proceso de unificación finalizado.")

---
Iniciando la unificación de maestros máximos de todos los años...

No se encontraron archivos de maestros máximos para unificar.
---
Iniciando la unificación de maestros mínimos de todos los años...

No se encontraron archivos de maestros mínimos para unificar.
---
Proceso de unificación finalizado.


# **5. Canvas de Proyectos de Data Science con Datos de Afiliados de EPS**

---

### **Sección 1: Análisis Descriptivo y de Calidad de Datos**
#### **Objetivo:** Entender la población de afiliados y la evolución de los datos.

* **Análisis de Afiliación en el SGSSS:**
    * **Pregunta a responder:** ¿Cómo ha evolucionado la afiliación de la EPS a lo largo del tiempo (por año y mes)?
    * **Métodos:** Contar registros únicos de afiliados por mes. Visualizar la tendencia en gráficos de líneas.
    * **Resultados esperados:** Gráficos de crecimiento/decrecimiento de la base de afiliados, identificación de picos o caídas en la afiliación.

* **Análisis de Calidad del Dato (BDUA):**
    * **Pregunta a responder:** ¿Cuál es la calidad de los datos de identificación y demográficos a lo largo de los años?
    * **Métodos:** Contar valores nulos o inconsistentes en campos clave como `HST_IDN_NUMERO_IDENTIFICACION_CF`, `AFL_FECHA_NACIMIENTO`, `DPR_ID`, `MNC_ID`.
    * **Resultados esperados:** Gráficos que muestren la evolución del porcentaje de datos faltantes, reportes de calidad que ayuden a identificar mejoras en la recolección de información.

---

### **Sección 2: Análisis Predictivo**
#### **Objetivo:** Predecir tendencias y eventos futuros para optimizar la gestión.

* **Modelo de Predicción de Desafiliación (Churn):**
    * **Pregunta a responder:** ¿Qué afiliados tienen mayor probabilidad de desafiliarse de la EPS en los próximos 6 meses?
    * **Métodos:** Usar algoritmos de clasificación (como Random Forest o Gradient Boosting) con variables como la antigüedad de la afiliación, cambios en el estado de cotización y características sociodemográficas.
    * **Resultados esperados:** Un modelo que clasifique a los afiliados en "alto riesgo", "riesgo moderado" y "bajo riesgo" de desafiliación.

* **Predicción de Pertenencia a Grupos Poblacionales:**
    * **Pregunta a responder:** ¿Cómo podemos predecir si un afiliado pertenece a un grupo étnico o poblacional específico basándonos en sus datos históricos?
    * **Métodos:** Aplicar modelos de clasificación para predecir la pertenencia a un `CNT_AFL_TPS_PRT_ETN_ID` o `TPS_GRP_PBL_ID`, utilizando variables como el lugar de residencia (`DPR_ID`, `MNC_ID`) y las características de su afiliación.
    * **Resultados esperados:** Un modelo que ayude a la EPS a identificar y validar de forma automática la pertenencia de los afiliados a grupos especiales.

---

### **Sección 3: Análisis Avanzado y de Tendencias**
#### **Objetivo:** Descubrir patrones complejos y relaciones no evidentes en los datos.

* **Detección de Anomalías:**
    * **Pregunta a responder:** ¿Existen registros de afiliados o patrones de afiliación inusuales que puedan indicar posibles fraudes o errores de digitación?
    * **Métodos:** Utilizar algoritmos de detección de anomalías (como Isolation Forest o DBSCAN) para identificar registros que no se ajustan al comportamiento de la mayoría.
    * **Resultados esperados:** Un sistema que alerte sobre registros con datos de identificación atípicos, fechas de nacimiento inverosímiles, o patrones de afiliación/desafiliación sospechosos.

* **Segmentación de la Población de Afiliados:**
    * **Pregunta a responder:** ¿Cómo podemos segmentar a los afiliados en grupos homogéneos para ofrecer servicios de salud más personalizados y eficientes?
    * **Métodos:** Aplicar algoritmos de clustering (como K-Means o jerárquico) usando variables demográficas (`AFL_FECHA_NACIMIENTO`, `TPS_GNR_ID`), de ubicación y de tipo de afiliación.
    * **Resultados esperados:** Segmentos de afiliados bien definidos (ej. "población joven urbana", "tercera edad rural") que permitan una planeación estratégica de servicios de salud.